In [1]:
import http.client
import urllib.request
import urllib.parse
import urllib.error
import base64
import json
import os

from IPython.display import Image
import cv2  
import pandas as pd
import numpy as np

## Load tagged from custom vision

In [2]:
headers = {
 # Request headers
 'Training-Key': '1a4a4c41bd394819be757c43ac8f1931' #--------------------แก้ตาม Custom Project
}
project_id = "5148bc53-cdbd-49ea-b302-216d183cb4a5" #--------------------แก้ตาม Custom Project

In [3]:
params = urllib.parse.urlencode({
 # Format - int32. Maximum number of images to return. Defaults to 50, limited to 256.
 'take': '50',
 # Format - int32. Number of images to skip before beginning the image batch. Defaults to 0.
 'skip': '0'
})

In [4]:
try:
     # base url
    conn = http.client.HTTPSConnection('southeastasia.api.cognitive.microsoft.com')
    conn.request("GET", f"/customvision/v3.0/training/projects/{project_id}/images/tagged?{params}", "{body}", headers)
    response = conn.getresponse() 
    # get response as a raw string
    data = response.read() 
    # convert the string to a json object
    data_json = json.loads(data)
except:
    print('error')

In [5]:
len(data_json)

10

## Dataframe for checking images

In [6]:
df = pd.read_csv('/home/yupaporn/codes/custom_labelling/Prostate.csv')
df.head(5)

,Unnamed: 0,Case,Num,FileName,Path,Subset
0,0,GE4586,2,GE4586_2.jpg,/media/tohn/HDD/VISION_dataset/Prostate/Train/...,Train
1,1,IE5235,1,IE5235_1.jpg,/media/tohn/HDD/VISION_dataset/Prostate/Train/...,Train
2,2,GI2946,0,GI2946_0.jpg,/media/tohn/HDD/VISION_dataset/Prostate/Train/...,Train
3,3,BK1461,8,BK1461_8.jpg,/media/tohn/HDD/VISION_dataset/Prostate/Train/...,Train
4,4,IV7824,0,IV7824_0.jpg,/media/tohn/HDD/VISION_dataset/Prostate/Train/...,Train


In [7]:
len(df)

161

In [8]:
thisdf=df

### Checking images loop 

#### รอบที่ 1 : 50

In [9]:
def findmat (thisdf):
    for record in thisdf.iterrows():
        img_path = record[1]['Path']
        img_c = cv2.imread(img_path)
        try:
            if (img_c.shape[0] == img.shape[0]) and (img_c.shape[1] == img.shape[1]):
                img_dif = np.sum(img-img_c)
                path = 'None'
                if img_dif == 0:
                    print(img_path)
                    path = img_path
                return path
        except:
            path = 'None'
            print('this is none')

In [10]:
datadf = list()
for i in range(0,len(data_json)):
    
    #load image origin save to test.jpg
    urllib.request.urlretrieve(data_json[i]['originalImageUri'],'test.jpg')
    img = cv2.imread('test.jpg') 
    
    originalImage = data_json[i]['originalImageUri']
    for j in range(len(data_json[i]['regions'])):    
        tagName = data_json[i]['regions'][j]['tagName']
        left = int(data_json[i]['regions'][j]['left'] * data_json[i]['width'])
        top = int(data_json[i]['regions'][j]['top'] * data_json[i]['height'])
        width = int(data_json[i]['regions'][j]['width'] * data_json[i]['width'])
        height = int(data_json[i]['regions'][j]['height'] * data_json[i]['height'])
    #     img = cv2.rectangle(img, (left,top), (left+width,top+height), (255,0,0), 5)
        #Real point
        Rleft =data_json[i]['regions'][j]['left']
        Rtop = data_json[i]['regions'][j]['top']
        Rwidth = data_json[i]['regions'][j]['width']
        Rheight = data_json[i]['regions'][j]['height']
    
        #loop check img
        print(i)
        path = findmat(thisdf)
        print(path)

        datadf.append({'img_path':path,'tagName':tagName,'originalImage': originalImage, 'left':left, 'top':top,
                              'width':width,'height':height, 'Rleft':Rleft, 'Rtop':Rtop,
                              'Rwidth':Rwidth,'Rheight':Rheight})

0
None
0
None
0
None
1
None
1
None
2
None
3
None
4
None
4
None
5
None
6
None
6
None
7
None
7
None
7
None
8
None
9
None
9
None
9
None


In [11]:
df = pd.DataFrame(datadf)
df.head()

,img_path,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,None,prostate,https://irisprodseatraining.blob.core.windows....,468,309,81,58,0.524609,0.430187,0.090604,0.081944
1,None,prostate,https://irisprodseatraining.blob.core.windows....,470,385,81,26,0.525727,0.535742,0.091723,0.036111
2,None,prostate,https://irisprodseatraining.blob.core.windows....,351,374,39,58,0.393736,0.520464,0.043624,0.080556
3,None,prostate,https://irisprodseatraining.blob.core.windows....,385,392,57,60,0.430804,0.544708,0.064732,0.083218
4,None,prostate,https://irisprodseatraining.blob.core.windows....,512,362,32,41,0.572545,0.503099,0.035714,0.056865


In [12]:
print(len(df))

19


In [13]:
os.chdir('/home/yupaporn/USAI_labeling/')
df.to_csv('Prostate_test.csv')